In [ ]:
! pip install -U kaleido && pip install git+https://github.com/jkminder/pycolors.git && pip install -e .../dictionary_learning

In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")
from transformers import AutoModelForCausalLM, AutoTokenizer
from dictionary_learning.cache import ActivationCache
from dictionary_learning.dictionary import CrossCoder
from datasets import load_from_disk, load_dataset
from loguru import logger
import torch as th
from nnsight import LanguageModel
from pathlib import Path
from pycolors import TailwindColorPalette
import os
from tools.feature_analysis import plot_feature_freq, plot_feature_diff
from pathlib import Path
from dlabutils import model_path

In [ ]:
cc_name = "l13-mu4.1e-02-lr1e-04"
stats_path = Path("../stats/")
save_dir = Path(f"../plots/") / model_name
base_dir = Path(f"../results/eval_crosscoder/{cc_name}_ae_final/data")

os.makedirs(save_dir, exist_ok=True)

stats_fineweb = th.load(stats_path / model_name / "fineweb.pt").to("cpu")
stats_lmsys = th.load(stats_path / model_name / "lmsys.pt").to("cpu")

In [ ]:
for rescaled in [True, False]:
    for split in ["joint", "instruction", "base"]:
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="base",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="instruction",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="shared",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name=None,
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )

In [ ]:
for rescaled in [True, False]:
    for group in ["base", "instruction", "shared", None]:
        plot_feature_diff(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name=group,
            rescaled=rescaled,
            indices_path=base_dir,
            save=True,
        )